<a href="https://colab.research.google.com/github/TienLungSun/LangChain/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

程式細節說明參考LangChain官網提供的Turtorial
https://python.langchain.com/v0.2/docs/tutorials/rag/

In [1]:
pip install langchain langchain_community langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:

In [2]:
pip install -qU langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.9 MB/s eta 0:00:00


下一行需要輸入你的OpenAI API key,初學者可以參考我的YT影片 https://youtu.be/WDvMPOeT8QA?si=8Wj3mDXAjcHwkxWd 申請API key

In [3]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

··········


測試直接送prompt給LLM (看看沒用RAG前的回應)

In [4]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content="誰演講AI?"),
]

llm.invoke(messages)

AIMessage(content='我不知道具體的演講者是誰，但許多專家和學者在不同的會議和論壇上討論人工智慧（AI）相關主題。您可以查找最近的AI會議或活動以了解更多。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 64, 'total_tokens': 116}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-9102a2c1-a36d-40d6-85bc-270b91028f92-0', usage_metadata={'input_tokens': 64, 'output_tokens': 52, 'total_tokens': 116})

#用 RAG 客製化LLM

跑下一行之前,先把pdf上傳到Colab,可以看我的YT教學影片 https://youtu.be/ly7aiVUyZFo?si=FoVJJ1o1PuIjmxB9

In [6]:
for filename in os.listdir():
  if filename.endswith(".pdf"):
    print(filename)

RAG資料檔-2024 上活動規劃.pdf
RAG資料檔-2023活動規劃.pdf


In [7]:
from langchain_community.document_loaders import PyPDFLoader

In [8]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 6.2 MB/s eta 0:00:00


In [9]:
all_docs = []
for filename in os.listdir():
  if filename.endswith(".pdf"):
    loader = PyPDFLoader(filename)
    docs = loader.load()
    for i in range(len(docs)):
      print(docs[i].metadata, end = ",")
    print()
    all_docs.extend(docs)

{'source': 'RAG資料檔-2024 上活動規劃.pdf', 'page': 0},{'source': 'RAG資料檔-2024 上活動規劃.pdf', 'page': 1},
{'source': 'RAG資料檔-2023活動規劃.pdf', 'page': 0},{'source': 'RAG資料檔-2023活動規劃.pdf', 'page': 1},


In [10]:
len(all_docs)

4

In [13]:
for i in range(len(all_docs)):
  print(all_docs[i].metadata)

{'source': 'RAG資料檔-2024 上活動規劃.pdf', 'page': 0}
{'source': 'RAG資料檔-2024 上活動規劃.pdf', 'page': 1}
{'source': 'RAG資料檔-2023活動規劃.pdf', 'page': 0}
{'source': 'RAG資料檔-2023活動規劃.pdf', 'page': 1}


In [14]:
all_docs[0].page_content[0:100]

'2024 Spring semester 半導體書院活動規劃  \n \n週數  Date  地點  活動類別  人數  時數  主題 /演講題目 /講者  備註  \n 1/18 (四) \n13:00~1'

##Split

依據文件特性,調整 chunk_size, chunk_overlap

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=100, add_start_index=True
)

all_splits = text_splitter.split_documents(all_docs)
print(len(all_splits))

7


In [17]:
for i in range(len(all_splits)):
  print(all_splits[i].metadata)

{'source': 'RAG資料檔-2024 上活動規劃.pdf', 'page': 0, 'start_index': 0}
{'source': 'RAG資料檔-2024 上活動規劃.pdf', 'page': 0, 'start_index': 393}
{'source': 'RAG資料檔-2024 上活動規劃.pdf', 'page': 0, 'start_index': 801}
{'source': 'RAG資料檔-2024 上活動規劃.pdf', 'page': 1, 'start_index': 0}
{'source': 'RAG資料檔-2023活動規劃.pdf', 'page': 0, 'start_index': 0}
{'source': 'RAG資料檔-2023活動規劃.pdf', 'page': 0, 'start_index': 392}
{'source': 'RAG資料檔-2023活動規劃.pdf', 'page': 1, 'start_index': 0}


##Store

In [18]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

##Retrieval

In [19]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("AI 影像辨識應用實務")
print(len(retrieved_docs))

6


In [20]:
for i in range(len(retrieved_docs)):
  print(retrieved_docs[i].metadata)

{'page': 1, 'source': 'RAG資料檔-2023活動規劃.pdf', 'start_index': 0}
{'page': 0, 'source': 'RAG資料檔-2023活動規劃.pdf', 'start_index': 0}
{'page': 1, 'source': 'RAG資料檔-2024 上活動規劃.pdf', 'start_index': 0}
{'page': 0, 'source': 'RAG資料檔-2024 上活動規劃.pdf', 'start_index': 801}
{'page': 0, 'source': 'RAG資料檔-2024 上活動規劃.pdf', 'start_index': 393}
{'page': 0, 'source': 'RAG資料檔-2023活動規劃.pdf', 'start_index': 392}


In [21]:
retrieved_docs = retriever.invoke("半導體")
for i in range(len(retrieved_docs)):
  print(retrieved_docs[i].metadata)

{'page': 0, 'source': 'RAG資料檔-2024 上活動規劃.pdf', 'start_index': 393}
{'page': 0, 'source': 'RAG資料檔-2023活動規劃.pdf', 'start_index': 0}
{'page': 0, 'source': 'RAG資料檔-2024 上活動規劃.pdf', 'start_index': 0}
{'page': 1, 'source': 'RAG資料檔-2023活動規劃.pdf', 'start_index': 0}
{'page': 0, 'source': 'RAG資料檔-2024 上活動規劃.pdf', 'start_index': 801}
{'page': 0, 'source': 'RAG資料檔-2023活動規劃.pdf', 'start_index': 392}


##Generation

In [22]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

print(example_messages)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:333: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langsmith/client.py:5431: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:")]

In [23]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [24]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("誰演講ＡＩ"):
    print(chunk, end="", flush=True)

10/4 的講者是台積電智能製造中心的林晏經理，演講主題為「半導體產業卓越智造」。

##到這邊已經完成RAG練習,以下是合併的寫法

把以上合併,直接寫chain

In [25]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "10/4 的講者是誰？"})
print(response["answer"])

有關AI的演講包括：10月18日的「AI的興起與半導體技術的變革」由旺宏電子的謝光宇處長主講，以及3月21日的「AI影像辨識應用實務」由品保處的歐筱華處長主講。


In [ ]:
for document in response["context"]:
    print(document.metadata)
    print()

{'page': 1, 'source': 'RAG資料檔-2023活動規劃.pdf', 'start_index': 0}

{'page': 0, 'source': 'RAG資料檔-2023活動規劃.pdf', 'start_index': 0}

{'page': 0, 'source': 'RAG資料檔-2024 上活動規劃.pdf', 'start_index': 801}

{'page': 1, 'source': 'RAG資料檔-2024 上活動規劃.pdf', 'start_index': 0}

{'page': 0, 'source': 'RAG資料檔-2024 上活動規劃.pdf', 'start_index': 0}

{'page': 0, 'source': 'RAG資料檔-2023活動規劃.pdf', 'start_index': 392}



##以下是用 prompt template 的寫法

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("誰演講ESG")

'演講ESG的是職安衛暨風險管理部的吳鴻明副處長。謝謝你的提問！'